# Notebook: Thu thập dữ liệu sản phẩm từ Pancake POS API

### 1.1. Import thư viện & Config

In [1]:
import os
import json
import time
import random
import requests
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import LONGTEXT
from sqlalchemy.types import BigInteger, DateTime
from dotenv import load_dotenv
from datetime import datetime

# Load biến môi trường
load_dotenv()

API_KEY   = os.getenv("API_KEY")
DB_USER   = os.getenv("DB_USER")
DB_PASS   = os.getenv("DB_PASS")
DB_HOST   = os.getenv("DB_HOST")
DB_PORT   = os.getenv("DB_PORT")
DB_BRONZE = os.getenv("DB_BRONZE")
SHOP_ID   = os.getenv("SHOP_ID")  # nhớ có SHOP_ID trong .env


### 1.2. Kết nối tới database Bronze

In [2]:
engine_bronze = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BRONZE}"
)

print(f"✅ Kết nối thành công tới Bronze: {DB_BRONZE}")


✅ Kết nối thành công tới Bronze: winner_bronze


### 1.3. Hàm lấy một trang dữ liệu sản phẩm từ API

In [3]:
def get_products_page(shop_id, page=1, page_size=1000, max_retries=3):
    """
    Gọi API /shops/{shop_id}/products để lấy một trang sản phẩm
    """
    url = f"https://pos.pages.fm/api/v1/shops/{shop_id}/products"
    params = {"api_key": API_KEY, "page": page, "page_size": page_size}
    
    for attempt in range(1, max_retries + 1):
        try:
            resp = requests.get(url, params=params, timeout=60)
            if resp.status_code == 200:
                j = resp.json()
                data = j.get("data", [])
                total_pages = j.get("total_pages") or j.get("meta", {}).get("total_pages")
                return data, total_pages
            else:
                print(f"⚠️ Lỗi {resp.status_code} tại page {page}, attempt {attempt}")
        except Exception as e:
            print(f"⚠️ Exception tại page {page}: {e}, attempt {attempt}")
        time.sleep(5 * attempt)  # backoff khi lỗi
    return [], None


### 1.4. Hàm ghi dữ liệu batch vào MySQL

In [4]:
from sqlalchemy.dialects.mysql import VARCHAR

def insert_products_batch(df_batch):
    df_batch.to_sql(
        "products_raw",
        con=engine_bronze,
        if_exists="append",
        index=False,
        dtype={
            "shop_id": BigInteger(),
            "product_id": VARCHAR(50),
            "raw_json": LONGTEXT(),
            "extracted_at": DateTime()
        }
    )


### 1.5. Hàm fetch & load nhiều trang khách hàng

In [5]:
def fetch_and_load_products(shop_id, start_page=1, page_size=1000):
    page = start_page
    while True:
        products, total_pages = get_products_page(shop_id, page, page_size)
        if not products:
            print("📦 Không còn dữ liệu tại page", page)
            break

        df_batch = pd.DataFrame([{
            "shop_id": shop_id,
            "product_id": prod.get("id"),
            "raw_json": json.dumps(prod, ensure_ascii=False),
            "extracted_at": datetime.now()
        } for prod in products])

        insert_products_batch(df_batch)
        print(f"✅ Page {page}/{total_pages} - Loaded {len(products)} products")

        if total_pages and page >= total_pages:
            break

        # Ngủ ngẫu nhiên để tránh bị chặn
        time.sleep(random.uniform(1, 3))
        page += 1


### 1.6. Chạy tiến trình thu thập dữ liệu

In [6]:
fetch_and_load_products(SHOP_ID, start_page=1, page_size=1000)


✅ Page 1/1 - Loaded 37 products
